In [134]:
import pandas as pd
import matplotlib.pyplot as plot
import requests as req
from census import Census
from us import states
import os

# Census API Key
c = Census("5e99f22848e97d5bcc53f03a358b973493473cd9")
cKey = "5e99f22848e97d5bcc53f03a358b973493473cd9"

In [135]:
# Read Main CSV
msgPpl = pd.read_csv("NCMEC_final.csv")
msgPpl.head()

,name,age,gender,ethnicity,NCMEC_case#,case_type,birth_year,missing_year,missing_city,missing_county,missing_state
0,DAVID BORER,36.0,male,white,722859,abduct,1981.0,1989.0,WILLOW,Matanuska Susitna,Alaska
1,JANE DOE1980,NaN,female,white,1184520,NaN,NaN,NaN,NaN,NaN,Alaska
2,MEGAN EMERICK,61.0,female,white,1080580,missing,1956.0,1973.0,SEWARD,Kenai Peninsula,Alaska
3,STELLA EVON,38.0,female,native american,919705,missing,1979.0,1996.0,BETHEL,Bethel,Alaska
4,AMY FANDEL,47.0,female,white,601234,abduct,1970.0,1978.0,STERLING,Kenai Peninsula,Alaska


In [136]:
# census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
#                           "B19301_001E",
#                           "B17001_002E"), {'for': 'zip code tabulation area:*'})

In [137]:
#Create census for states
url = "https://api.census.gov/data/2014/pep/natstprc?get=STNAME,POP&for=state:*&DATE=7"
reqUrl = url + "&key="+ cKey
getData = req.get(reqUrl).json()
df_state_pop = pd.DataFrame(getData,columns=getData[0])
df_state_pop = df_state_pop.drop(df_state_pop.index[0])
df_state_pop['POP'] =  list(map(int,df_state_pop['POP']))
df_state_pop = df_state_pop[df_state_pop.STNAME != 'District of Columbia']
df_state_pop = df_state_pop[df_state_pop.STNAME != 'Puerto Rico Commonwealth']
df_state_pop = df_state_pop.reset_index(drop = True)
msgState = msgPpl.groupby("missing_state").count()["name"]
df_state_pop['Missing_Population'] = list(msgState)
df_state_pop['Missing_Ratio'] = round(df_state_pop['Missing_Population']/df_state_pop['POP'] * 1000000,2)
# df_state_pop.loc[df_state_pop['Missing_Ratio'] > 20]
df_state_pop

,STNAME,POP,DATE,state,Missing_Population,Missing_Ratio
0,Alabama,4849377,7,01,38,7.84
1,Alaska,736732,7,02,13,17.65
2,Arizona,6731484,7,04,152,22.58
3,Arkansas,2966369,7,05,40,13.48
4,California,38802500,7,06,620,15.98
5,Colorado,5355866,7,08,87,16.24
6,Connecticut,3596677,7,09,25,6.95
7,Delaware,935614,7,10,10,10.69
8,Florida,19893297,7,12,360,18.10
9,Georgia,10097343,7,13,98,9.71


In [138]:
dfbottom3State = df_state_pop.sort_values(["POP"], ascending = True).head(3)
dfbottom3State

,STNAME,POP,DATE,state,Missing_Population,Missing_Ratio
49,Wyoming,584153,7,56,7,11.98
44,Vermont,626562,7,50,8,12.77
1,Alaska,736732,7,02,13,17.65


In [139]:
dfTop3State = df_state_pop.sort_values(["POP"], ascending=False).head(3)
dfTop3State

,STNAME,POP,DATE,state,Missing_Population,Missing_Ratio
4,California,38802500,7,06,620,15.98
42,Texas,26956958,7,48,368,13.65
8,Florida,19893297,7,12,360,18.10


In [147]:
# Create an income list by state
# https://api.census.gov/data/2015/acs1?get=NAME,B19013_001E&for=state:*%20&key=5e99f22848e97d5bcc53f03a358b973493473cd9
url = "https://api.census.gov/data/2015/acs1?get=NAME,B19013_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B02001_009E,B02001_010E&for=state:*"
incUrl = url + "&key="+ cKey
getIncData = req.get(incUrl).json()
getIncData = pd.DataFrame(getIncData, columns=getIncData[0])
getIncData = getIncData.drop(getIncData.index[0])
# getIncData["Total Ethnic"] =  int (getIncData["B02001_002E"]) + int (getIncData["B02001_003E"]) +int (getIncData["B02001_004E"]) + int (getIncData["B02001_005E"]) + int (getIncData["B02001_006E"]) + int (getIncData["B02001_007E"]) + int (getIncData["B02001_008E"]) + int (getIncData["B02001_009E"]) + int (getIncData["B02001_010E"])
getIncData = getIncData.rename(columns={"B19013_001E": "Household Income", 
                                      "B02001_002E": "Whites", 
                                      "B02001_003E": "African Americans",
                                      "B02001_004E": "American Indians",
                                      "B02001_005E": "Asians",
                                      "B02001_006E": "Pacific Islanders",
                                      "B02001_007E": "Others",
                                      "B02001_008E": "Two or More Races",
                                      "B02001_009E": "Two or More Inc Others",
                                      "B02001_010E": "Two or More Excl Others",
                                      "NAME": "STNAME" })
getIncData

,STNAME,Household Income,Whites,African Americans,American Indians,Asians,Pacific Islanders,Others,Two or More Races,Two or More Inc Others,Two or More Excl Others,state
1,Alabama,44765,3329913,1303298,20503,59393,4448,54105,87319,6354,80965,01
2,Alaska,73355,481314,26176,101133,47295,6762,12467,63285,3171,60114,02
3,Arizona,51492,5285994,300685,308469,217556,11284,481631,222446,45251,177195,04
4,Arkansas,41995,2308047,469555,20797,42404,7639,65542,64220,6149,58071,05
5,California,64500,23824254,2277229,282777,5548936,157554,5300297,1753771,375682,1378089,06
6,Colorado,63909,4607390,224809,49138,162880,9136,209890,193331,30981,162350,08
7,Connecticut,71346,2747943,380571,8893,156249,519,180107,116604,28025,88579,09
8,Delaware,61255,654904,203863,3121,36114,334,19157,28441,3500,24941,10
9,District of Columbia,75628,269143,318831,1661,25944,475,34797,21377,3583,17794,11
10,Florida,49426,15357374,3280778,47032,544068,11898,528266,501856,87649,414207,12


In [148]:
getIncData = getIncData[getIncData.STNAME != 'District of Columbia']
getIncData = getIncData[getIncData.STNAME != 'Puerto Rico Commonwealth']
getIncData = getIncData.reset_index(drop = True)
getIncData

,STNAME,Household Income,Whites,African Americans,American Indians,Asians,Pacific Islanders,Others,Two or More Races,Two or More Inc Others,Two or More Excl Others,state
0,Alabama,44765,3329913,1303298,20503,59393,4448,54105,87319,6354,80965,01
1,Alaska,73355,481314,26176,101133,47295,6762,12467,63285,3171,60114,02
2,Arizona,51492,5285994,300685,308469,217556,11284,481631,222446,45251,177195,04
3,Arkansas,41995,2308047,469555,20797,42404,7639,65542,64220,6149,58071,05
4,California,64500,23824254,2277229,282777,5548936,157554,5300297,1753771,375682,1378089,06
5,Colorado,63909,4607390,224809,49138,162880,9136,209890,193331,30981,162350,08
6,Connecticut,71346,2747943,380571,8893,156249,519,180107,116604,28025,88579,09
7,Delaware,61255,654904,203863,3121,36114,334,19157,28441,3500,24941,10
8,Florida,49426,15357374,3280778,47032,544068,11898,528266,501856,87649,414207,12
9,Georgia,51244,6077438,3197929,28491,388457,4337,284084,234124,36148,197976,13


In [151]:
allDataDf = df_state_pop.merge(getIncData)

In [153]:
#Create graphs


0            Alabama
1             Alaska
2            Arizona
3           Arkansas
4         California
5           Colorado
6        Connecticut
7           Delaware
8            Florida
9            Georgia
10            Hawaii
11             Idaho
12          Illinois
13           Indiana
14              Iowa
15            Kansas
16          Kentucky
17         Louisiana
18             Maine
19          Maryland
20     Massachusetts
21          Michigan
22         Minnesota
23       Mississippi
24          Missouri
25           Montana
26          Nebraska
27            Nevada
28     New Hampshire
29        New Jersey
30        New Mexico
31          New York
32    North Carolina
33      North Dakota
34              Ohio
35          Oklahoma
36            Oregon
37      Pennsylvania
38      Rhode Island
39    South Carolina
40      South Dakota
41         Tennessee
42             Texas
43              Utah
44           Vermont
45          Virginia
46        Washington
47     West V